In [1]:
import os
import sys
import copy
import pickle
import concurrent.futures
import numpy as np
import pandas as pd

import cantera as ct
import rmgpy.chemkin

sys.path.append('/work/westgroup/harris.se/autoscience/autoscience_workflow/workflow/scripts/kinetics')
import kineticfun


import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# Load the base model
basedir = '/work/westgroup/harris.se/autoscience/autoscience/butane/sensitivity_analysis/base_rmg24'

base_chemkin = os.path.join(basedir, 'chem_annotated.inp')
dictionary = os.path.join(basedir, 'species_dictionary.txt')
transport = os.path.join(basedir, 'tran.dat')
species_list, reaction_list = rmgpy.chemkin.load_chemkin_file(base_chemkin, dictionary_path=dictionary, transport_path=transport)
print(f'{len(reaction_list)} reactions and {len(species_list)} species')

1556030143.py:7 <module> INFO Thermo file has default temperature range 300.0 to 1000.0 and 1000.0 to 5000.0


1822 reactions and 110 species


In [3]:
base_flame_speed = 0.41552749

In [6]:
flame_speeds_rxns = base_flame_speed * np.ones(len(reaction_list))
flame_speeds_sp = base_flame_speed * np.ones(len(species_list))

In [7]:
# load individual flame speed files
results_dir = '/work/westgroup/harris.se/autoscience/autoscience/butane/sensitivity_analysis/base_rmg24/results/flame_speeds'
for i in range(0, len(reaction_list)):
    result_file = os.path.join(results_dir, f'flame_speed_{i:04}.txt')
    if not os.path.exists(result_file):
        continue
    with open(result_file, 'r') as f:
        result = f.read()
        flame_speeds_rxns[i] = result
        
for i in range(0, len(species_list)):
    j = i + len(reaction_list)
    result_file = os.path.join(results_dir, f'flame_speed_{j:04}.txt')
    if not os.path.exists(result_file):
        continue
    with open(result_file, 'r') as f:
        result = f.read()
        flame_speeds_sp[i] = result

In [12]:
# sort the reaction list by difference in flame speed
order_rxn = np.arange(0, len(flame_speeds_rxns))
fs_diff = np.abs(flame_speeds_rxns - base_flame_speed)
most_sensitive_rxn = [x for _, x in sorted(zip(fs_diff, order_rxn))][::-1]


order_sp = np.arange(0, len(flame_speeds_sp))
fs_diff = np.abs(flame_speeds_sp - base_flame_speed)
most_sensitive_sp = [x for _, x in sorted(zip(fs_diff, order_sp))][::-1]

In [19]:
flame_speeds_sp[17]

0.2256325676932963

In [18]:
flame_speeds_sp[85]

0.4155011033078316

In [16]:
# print the top species
for i in range(0, 50):
    
    index = most_sensitive_sp[i]
    
    print(index, '\t', species_list[index])

17 	 H(14)
10 	 CO2(7)
8 	 O(5)
9 	 CO(6)
26 	 CH2(23)
21 	 CH3(18)
11 	 H2O(8)
25 	 C2H3(22)
63 	 C4H71-3(190)
4 	 butane(1)
6 	 CH(3)
18 	 OH(15)
66 	 C4H6(194)
5 	 O2(2)
14 	 C2H4(11)
57 	 SC4H9(183)
20 	 H2O2(17)
16 	 H2(13)
61 	 C4H8-1(188)
97 	 C=CC(C)O[O](4243)
62 	 C4H8-2(189)
32 	 C2H5(33)
59 	 SC4H9O2(186)
50 	 C3H5-A(94)
7 	 C2H(4)
28 	 C2H2(25)
31 	 C2H6(32)
15 	 C3H6(12)
83 	 C[C]CC-2(776)
12 	 CH2O(9)
64 	 C4H71-4(191)
74 	 C4H8OOH2-4O2(229)
33 	 CH3CHO(35)
60 	 SC4H9O2H(187)
98 	 CC=CCO[O](4244)
39 	 C2H5O2H(48)
45 	 CH3COCH2O2(72)
22 	 HCO(19)
37 	 CH3O2H(46)
47 	 C2H3CHO(74)
70 	 C4H8OOH1-3(219)
27 	 CH2CO(24)
73 	 C4H8OOH1-3O2(225)
13 	 CH4(10)
38 	 C2H5O2(47)
35 	 HOCHO(40)
72 	 C4H8OOH2-4(223)
58 	 PC4H9O2(184)
29 	 CH3OH(30)
30 	 CH3O(31)


In [17]:
# print the top reactions
for i in range(0, 50):
    
    
    index = most_sensitive_rxn[i]
    
    print(index, '\t', reaction_list[index])
    
#     if index > len(reaction_list):
#         index = index - len(reaction_list)
#         print(index, '\t', species_list[index])
#     else:
#         try:
#             if reaction_list[index].family != 'H_Abstraction' and \
#                 reaction_list[index].family != 'Disproportionation' and \
#                 reaction_list[index].family != 'intra_H_migration' and \
#                 reaction_list[index].family != 'R_Addition_MultipleBond':
#                 continue
#             print(index, '\t', reaction_list[index], '\t', reaction_list[index].family)
#         except AttributeError:
#             continue
#             print(index, '\t', reaction_list[index], '\t', 'PDEP')

0 	 O2(2) + H(14) <=> O(5) + OH(15)
78 	 O(5) + C2H2(25) <=> CO(6) + CH2(23)
179 	 H(14) + HCO(19) <=> H2(13) + CO(6)
199 	 CH3(18) + CH3(18) <=> H(14) + C2H5(33)
113 	 OH(15) + HCO(19) <=> H2O(8) + CO(6)
12 	 H2O(8) + H2O(8) <=> H(14) + OH(15) + H2O(8)
121 	 O2(2) + H(14) + H2O(8) <=> HO2(16) + H2O(8)
1262 	 HO2(16) + C4H71-3(190) <=> OH(15) + C4H7O(202)
1263 	 CH3O2(45) + C4H71-3(190) <=> CH3O(31) + C4H7O(202)
184 	 H(14) + CH2O(9) <=> H2(13) + HCO(19)
2 	 OH(15) + H2(13) <=> H(14) + H2O(8)
246 	 H(14) + butane(1) <=> H2(13) + SC4H9(183)
697 	 H(14) + C4H71-3(190) <=> C4H8-1(188)
180 	 H2(13) + CH(3) <=> H(14) + CH2(23)
706 	 H(14) + C4H71-3(190) <=> C4H8-2(189)
34 	 O(5) + C2H4(11) <=> CH2(23) + CH2O(9)
185 	 O(5) + CH3(18) <=> H(14) + H2(13) + CO(6)
116 	 OH(15) + CH3(18) <=> H2O(8) + CH2(23)
556 	 OH(15) + C3H6(12) <=> H2O(8) + C3H5-A(94)
558 	 H(14) + C3H6(12) <=> H2(13) + C3H5-A(94)
65 	 O2(2) + C2H3(22) <=> O(5) + CH2CHO(21)
136 	 O2(2) + C2H3(22) <=> HCO(19) + CH2O(9)
15 	 H(1